### **API 키 설정**

In [ ]:
from dotenv import load_dotenv
load_dotenv()

### **필요 라이브러리 호출**

In [1]:
from datetime import datetime
from typing import Annotated, TypedDict
from pydantic import BaseModel, Field

from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

from openai import OpenAI

from langchain_experimental.utilities import PythonREPL
from langchain_community.tools.tavily_search import TavilySearchResults

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

### **이미지 생성 도구 정의**

In [2]:
client = OpenAI()

# DALL-E 이미지 생성을 위한 스키마 정의
class GenImageSchema(BaseModel):
    prompt: str = Field(description="The prompt for image generation")

# DALL-E 이미지 생성 함수 정의
@tool(args_schema=GenImageSchema)
def generate_image(prompt: str) -> str:
    """
    Generate an image using DALL-E based on the given prompt.
    """
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="standard",
        n=1
    )
    return response.data[0].url

### **코드 실행 도구 정의**

In [3]:
repl = PythonREPL()

@tool
def python_repl(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code.
    This is visible to the user. Please use ploly to visualize."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return result_str

### **에이전트 정의**

In [7]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

system_prompt = f"""
Today is {datetime.now().strftime("%Y-%m-%d")}
You are a helpful AI Assistant that can use web search tool(tavily ai api), image generation tool(DallE API) and code execution tool(Python REPL).
You should always answer in same language as user's ask.
When user ask about the information that you can't answer, you can call the search tool.
When user ask about generating image, you can call the generate_image tool.
When user ask about Data analysis, data visualization or code execution image, you can call the python repl tool.
"""

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template("{input}")
])

search = TavilySearchResults(max_results=5)

tools = [search, generate_image, python_repl]
llm_with_tools = llm.bind_tools(tools)

chain = prompt | llm_with_tools

In [8]:
result = chain.invoke("대한민국 GDP 추이를 그래프로 그려줘")
print(result)

content='' additional_kwargs={'tool_calls': [{'id': 'call_I2ETKVc99FQd5AhQp8Xb21HT', 'function': {'arguments': '{"query":"대한민국 GDP 추이 2025"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 282, 'total_tokens': 308, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_432e014d75', 'id': 'chatcmpl-BLA7uDhII7M44jUfZnyNwax0AOgOC', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-095c4c0e-7442-4935-9a61-46841a6b4304-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '대한민국 GDP 추이 2025'}, 'id': 'call_I2ETKVc99FQd5AhQp8Xb21HT', 'type': 'tool_call'}] usage_metadata={'input_tokens': 282, 'output_tokens': 26, 'total_tokens': 308, 'input_token_details'

In [9]:
result.additional_kwargs['tool_calls']

[{'id': 'call_I2ETKVc99FQd5AhQp8Xb21HT',
  'function': {'arguments': '{"query":"대한민국 GDP 추이 2025"}',
   'name': 'tavily_search_results_json'},
  'type': 'function'}]

### **그래프 구축**

In [10]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State):
    return {"messages": [chain.invoke(state["messages"])]}

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)

# 도구 노드 추가
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

# 조건부 엣지 추가
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# 도구 호출 후 채팅봇으로 돌아가는 엣지 추가
graph_builder.add_edge("tools", "chatbot")

# 시작점 설정
graph_builder.add_edge(START, "chatbot")

# 그래프 컴파일
graph = graph_builder.compile()

In [11]:
# The config is the **second positional argument** to stream() or invoke()!
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [("user", input("User: "))]}, config, stream_mode="values"
)

for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hello
================================== Ai Message ==================================

Hello! How can I assist you today?
